In [1]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.6 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 2.12.0
    Uninstalling openai-2.12.0:
      Successfully uninstalled openai-2.12.0


In [2]:
!pip install sentence-transformers faiss-cpu openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 42.2 MB/s eta 0:00:00


In [ ]:
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
import openai
from typing import List

class SimpleRAG:
    def __init__(self, api_key: str):
        self.encoder = SentenceTransformer('all-MiniLM-L6-v2')
        openai.api_key = api_key
        self.documents = []
        self.index = None
        self.embeddings = None

    def add_documents(self, documents: List[str]):
        """Добавляем документы в базу знаний и перестраиваем индекс"""
        self.documents.extend(documents)

        # Генерируем эмбеддинги для всех документов
        self.embeddings = self.encoder.encode(self.documents)
        dimension = self.embeddings.shape[1]

        # Создаем новый индекс FAISS
        self.index = faiss.IndexFlatIP(dimension)

        # Нормализуем векторы для косинусного сходства
        faiss.normalize_L2(self.embeddings)
        self.index.add(self.embeddings.astype('float32'))

    def search(self, query: str, k: int = 3) -> List[str]:
        """Поиск k наиболее релевантных документов"""
        if self.index is None or len(self.documents) == 0:
            return []

        query_embedding = self.encoder.encode([query])
        faiss.normalize_L2(query_embedding)

        # Поиск в индексе
        scores, indices = self.index.search(query_embedding.astype('float32'), k)

        # Возвращаем документы по индексам
        return [self.documents[idx] for idx in indices[0]]

    def generate_answer(self, query: str, context: List[str]) -> str:
        """Генерация ответа на основе контекста"""
        if not context:
            return "Не найдено релевантной информации для ответа на вопрос."

        context_text = '\n'.join([f'Документ {i+1}: {doc}' for i, doc in enumerate(context)])

        prompt = f'''Контекст:
{context_text}

Вопрос: {query}

Ответь на вопрос, используя только информацию из предоставленного контекста.
Если ответа нет в контексте, напиши "Информация не найдена в предоставленных документах".'''

        try:
            response = openai.ChatCompletion.create(
                model='gpt-3.5-turbo',
                messages=[{'role': 'user', 'content': prompt}],
                max_tokens=200,
                temperature=0.1
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            return f"Ошибка при генерации ответа: {str(e)}"

    def ask(self, query: str) -> dict:
        """Основная функция: поиск + генерация ответа"""
        relevant_docs = self.search(query)
        answer = self.generate_answer(query, relevant_docs)

        return {
            "query": query,
            "answer": answer,
            "sources": relevant_docs
        }


# Использование
if __name__ == "__main__":
    rag = SimpleRAG(api_key='rHeBELsvkrhy_Bj6tRQA38a37VeMaF2T3BlbkFJnR9rsWfl0x_fcdQdDSohhWHU8KRzEFwm0BFZ04d3qoCBc57a9mEE868gIdEHWxX9KPLX062AA')

    documents = [
        'Python - высокоуровневый язык программирования общего назначения. Создан в 1991 году Гвидо ван Россумом.',
        'RAG - архитектурный подход, который объединяет поиск информации с генерацией текста.',
        'Векторные базы данных хранят данные в виде многомерных векторов и позволяют выполнять семантический поиск.',
        'FAISS - библиотека для эффективного поиска схожих векторов.',
        'Эмбеддинги - это векторные представления текста, которые кодируют семантическое значение в числовой форме.'
    ]

    rag.add_documents(documents)

    questions = [
        'Кто создал Python?',
        'Что такое RAG?',
        'Как работают векторные базы данных?',
        'Что такое FAISS?'
    ]

    for question in questions:
        result = rag.ask(question)
        print(f"Вопрос: {result['query']}")
        print(f"Ответ: {result['answer']}")
        print(f"Источники: {result['sources']}")
        print("-" * 50)